# A Sketch using the new sambuca-core functionality

This is a sketch of one way to perform Sambuca parameter estimation using the sambuca-core functionality.

I prepared this notebook for two reasons: first, as a design sketch for functionality that I plan to implement in the actual sambuca package, and second as reference notes for other users of the system.

# Set up the environment
Nothing fancy here, just some imports and other setup code.

First, import the standard Scipy stuff, and configure matplotlib

In [1]:
%matplotlib inline
from collections import namedtuple
from pkg_resources import resource_filename
from os.path import join

import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.optimize import minimize

# set some controls on numpy formatting
# 5 decimal places, suppress scientific notation
np.set_printoptions(precision=5, suppress=True)

# set the matplotlib style to emulate ggplot2 from R
plt.style.use('ggplot')
plot_width = 12
plot_height = plot_width * 3/4

Now import the sambuca and sambuca-core packages:

In [2]:
import sambuca as sb
import sambuca_core as sbc

# Utility Functions

In [3]:
plot_items = []

In [4]:
def print_parameters(p):
    print(
'''\
    CHL:  {0:10.5f}
    CDOM: {1:10.5f}
    TR:   {2:10.5f}
    H:    {3:10.5f}
    Q:    {4:10.5f}'''
          .format(p.chl,p.cdom,p.nap,p.depth,p.substrate_fraction))

In [5]:
def show_plot():
    plt.figure(figsize=(plot_width, plot_height))
    for label, data in plot_items:
        plt.plot(data[0], data[1], label=label)
        
    plt.legend(loc='upper right')
    plt.show()

In [6]:
def add_sensor_filter_to_plot(filter_data):
    band_centre_wavelengths = filter_data[0]
    sensor_filter = filter_data[1]
    num_bands = sensor_filter.shape[0]

    for band in range(num_bands):
        band_data = sensor_filter[band, :]
        plot_items.append((
                'Filter Band {0}'.format(band),
                (band_centre_wavelengths, band_data)))

# Load the reference data

## Parameter bounds
Define the upper and lower bounds for the free parameters.

In [7]:
p_min = sb.FreeParameters(
    chl=0.01, 
    cdom=0.0005, 
    nap=0.2,
    depth=0.1,
    substrate_fraction=0)

In [8]:
p_max = sb.FreeParameters(
    chl=0.22, 
    cdom=0.015, 
    nap=2.4,
    depth=17.4,
    substrate_fraction=1)

## observed reflectance

## Load SIOPs and other data using the sambuca-core functions

In [ ]:
base_path = '/home/dc/code/sambuca-project/bioopti_data/'

### Load the sensor filter:

In [ ]:
sensor_filters = sbc.load_sensor_filters(join(base_path, 'sensor_filters'))
sensor_filters.keys()

In [ ]:
for k,v in sensor_filters.items():
    print(k, v[1].shape)

In [ ]:
sensor_filter = sensor_filters['ALOS']
sensor_filter[1].shape

In [ ]:
plot_items.clear()
add_sensor_filter_to_plot(sensor_filter)
show_plot()

### Load the substrates
This example uses the load_all_spectral_libraries function to return a dictionary of everything found in a directory. We then retrieve the required substrates from the dictionary.

In [ ]:
all_substrates = sbc.load_all_spectral_libraries(join(base_path, 'Substrates'))
substrate1 = all_substrates['moreton_bay_speclib:white Sand']
substrate2 = all_substrates['moreton_bay_speclib:brown Mud']

Note that the sambuca_core spectral library loading functions return a dictionary of (band-centre wavelength, value) tuples. Thus substrate1 and substrate2 are tuples.

### Aphystar
Here we load a single file, although we could have used the load_all_spectral_libraries approach with the same result.

Note that although we specify a single file, a dictionary is still returned. This is for two reasons: consistency with the other functions, and because a single file may contain multiple spectra.

In [ ]:
aphy_star = sbc.load_spectral_library(join(base_path, 'SIOP/WL08_aphy_1nm.hdr'))\
['wl08_aphy_1nm:WL08_aphy_star_mean_correct.csv:C2']

### Awater

In [ ]:
awater = sbc.load_spectral_library(join(base_path, 'SIOP/aw_350_900_lw2002_1nm.csv')).popitem()[1]

### Plot the SIOPS, just because we can :)

In [ ]:
# Leave awater out of the first plot
plot_items.clear()
plot_items.append(('s1', substrate1))
plot_items.append(('s2', substrate2))
plot_items.append(('aphy*', aphy_star))
add_sensor_filter_to_plot(sensor_filter)
show_plot()

plot_items.append(('awater', awater))
show_plot()

# Define the objective function builder

In [ ]:
def make_objective(
    awater,
    wavelengths,
    aphy_star,
    substrate1,
    substrate2,
    sensor_filter, 
    obs_spectra, 
    p_bounds=None, 
    noise=None):
    
    num_modelled_bands = sensor_filter.shape[1]
    num_observed_bands = sensor_filter.shape[0]
            
    def objective(p):
        '''
        Returns an objective score for the given parameter set.
        
        Args:
            p (tuple): The parameter tuple (chl, cdom, nap, substrate_fraction, depth).
        '''
        # To support algorithms without support for boundary values, we assign a high 
        # score to out of range parameters. This may not be the best approach!!!
        # p_bounds is a tuple of (min, max) pairs for each parameter in p
        if p_bounds is not None:
            for _p, lu in zip(p, p_bounds):
                l, u = lu
                if _p < l or _p > u:
                    return 100000.0
                    
        # call the forward model, relying on the default values for other inputs
        model_results = sbc.forward_model(
            chl = p[0],
            cdom = p[1],
            nap = p[2],
            depth = p[4],
            substrate_fraction = p[3],
            substrate1=substrate1,
            substrate2=substrate2,
            wavelengths=wav,
            a_water=awater,
            a_ph_star=aphy_star,
            num_bands=num_modelled_bands)
        
        # Apply the sensor filter
        filtered_spectra = sbc.apply_sensor_filter(model_results.rrs, sensor_filter)
        
        # Calculate the error and return it as the objective score
        error = sbc.error_all(obs_spectra, filtered_spectra, noise)
        
        #return error.alpha
        #return error.alpha_f
        return error.f
        #return error.lsq
    
    return objective